# Data Gathering Cleanup

In [1]:
import datetime as dt
import pandas as pd
import pandas_datareader.data as web
from scipy import stats
import numpy as np
import time
from itertools import chain
from functools import reduce

##### ETF Data Clean up ###########
class ETFDataCleanup(object):
    
    def __init__(self):
        pass

    def showNaColumns(self,df):
        s=df.isnull().sum()
        print(s[s>0])
    
    def dropNAColumns(self,df):
        return df.dropna(axis='columns')
    
    def computeDailyReturns(self,df):
        return df.pct_change().dropna()        
    
    
####### Get Data for Constituents of ETF
class ConstituentsData(ETFDataCleanup):
    
    def __init__(self):
        self.constituentdata=[]
        self.constituentcloseDF=[]
        self.tickerdf=[]
        self.changeDF=[]
        self.weightedStockReturns=[]
    
    def loadETFWeights(self,fileName):
        self.etfWeights = pd.read_excel(fileName)
        self.etfWeights.set_index('Ticker',inplace=True)

    def getStockPrices(self,startdate,enddate):
        tickers=self.etfWeights.index
        self.constituentdata =  web.DataReader(tickers,'yahoo',startdate,enddate)
        self.constituentcloseDF = self.constituentdata['Close'].iloc[:, :]
        
    def stringWeightsToFloat(self):
        self.etfWeights['Weights']=self.etfWeights['Weights'].apply(lambda x: x.replace('%','')).astype(float)
    
    def findNetAssetValue(self):
        self.weightedStockReturns=self.changeDF.copy()
        for col in self.changeDF.columns:
            # Divide by 100 for weights percentage eg 23.28%
            self.weightedStockReturns[col]=self.changeDF[col]*self.etfWeights['Weights'].loc[col]/100
        self.weightedStockReturns['NAV']=self.weightedStockReturns.sum(axis=1)

####### Get prices of ETF        
class ETFStockPrices(ETFDataCleanup):
    
    def __init__(self,etfticker=None):
        self.etfticker=etfticker
        self.etfdata=[]
        self.etfchangeDF=[]

    def getETFTickerData(self,startdate,enddate):
        self.etfdata =  web.DataReader(self.etfticker,'yahoo',startdate,enddate)







# Arbitrage Analysis

In [2]:
import pandas as pd
import numpy as np
from scipy import stats


###### Do the Arbitrage analysis work
class ETFArbitrage(object):

    def __init__(self,etfob,weightedStockReturns):
        self.navDF=pd.merge(etfob,weightedStockReturns['NAV'],left_index=True,right_index=True)
        self.navDF['Date']=self.navDF.index
        self.navDF['Close']=self.navDF['Close']*100
        self.navDF['NAV']=self.navDF['NAV']*100
        del self.navDF['Date']
        self.navDF['Mispricing']=(self.navDF['Close']-self.navDF['NAV'])

    
class StatisticalCalculations():

    def __init__(self):
        pass

    def findZScore(self,df=None,colname=None):
        # Check if the object is not a pandas dataframe, if not convert it
        if isinstance(df,pd.Series):
            df=df.to_frame()
            df.columns=[colname]
        df[colname + ' Z-Score']=np.abs(stats.zscore(df[colname].tolist())).round(2)
        return df

    def getDataAboveThresold(self,df,zthresh=None):
        df=df[df['Z-Score']>zthresh]
        return df

    def invertDict(self,d):
        newdict = {}
        for key, value in d.items():
            for string in value:
                newdict.setdefault(string, []).append(key)
        return newdict

class ArbitrageAnalysis(object):

    def __init__(self):
        pass

    def GetArbDataFrame(self,tickers=None,constituentdata=None,changeDF=None,daysofarbitrage=None,stdthresold=None):
        # Making Sure that Both ChangeDF and constituentdata has same Number of Dates, Since we dropped NA in change -KTZ
        tickersSTDdata={}
        statOb=StatisticalCalculations()
        daysofarbitrage=statOb.findZScore(df=daysofarbitrage,colname='Mispricing')

        for ticker in tickers:
            tickersSTDdata[ticker]=self.getConstituentsStdData(ticker,changeDF,constituentdata,statOb)

        print(tickersSTDdata)
        print(daysofarbitrage)
        return daysofarbitrage, tickersSTDdata

    def getConstituentsStdData(self,ticker,changeDF,constituentdata,statOb):
        weightedMovement=changeDF[ticker]*constituentdata['Volume'][ticker]
        weightedMovement=weightedMovement.dropna()

        stockVolumeStd=statOb.findZScore(df=constituentdata['Volume'][ticker],colname='Vol.')
        stockReturnStd=statOb.findZScore(df=changeDF[ticker]*100,colname='Return')
        stockweightedmovement=statOb.findZScore(df=weightedMovement,colname='Vol. Wgt Return')
        df=[stockVolumeStd,stockReturnStd,stockweightedmovement]
        df_merged = reduce(lambda  left,right: pd.merge(left,right,left_index=True,right_index=True,how='outer'), df)

        return df_merged[['Vol. Z-Score','Return Z-Score','Vol. Wgt Return Z-Score']].dropna()


# Jinga Template Display

In [3]:
class CleanDataForJinga(object):

    def __init__(self,data,columnRearrange,indexColumnName,reverse=None):
        self.data=data
        self.columnRearrange=columnRearrange
        self.indexColumnName=indexColumnName
        self.reverse=reverse

    def CleanForEndUser(self):
        print(self.data)
        print(self.indexColumnName)
        print(self.data.index)
        # Give index as one of the column name and deleted index
        self.data[self.indexColumnName]=self.data.index
        
        # Rearrange Column Names as desired to be displayed
        self.data=self.data[self.columnRearrange]

        # Remove the index from the dataframe
        self.data=self.data.reset_index(drop=True)

        # Normalize Dates from '2020-01-01 00:00:00' to '2020-01-01' in pandas
        if self.indexColumnName=='Date':
            self.data['Date'] = self.data['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))

        # Check if Dataframe needs to be reversed
        if self.reverse:
            # Reverse the DataFrame as needed
            self.data=self.data.iloc[::-1]

        return self.data





# Main Handle

In [4]:
tickeretf = 'XLF'
stdthresold = float(2)

print("Ticker is = "+tickeretf)
print("Tracking Error is = "+str(stdthresold))

filename='ETFDailyData'+'/'+dt.datetime.now().strftime("%Y%m%d")+'/'+tickeretf+'.xls'
startdate=dt.datetime(2019,1,1)
enddate=dt.datetime.today()

###$$$ Getting ticker prices, cleaning them and finding NET Asset Valu
# Create an object of Constituents Data of ETF - ConstituentsData
ob=ConstituentsData()
# Load ETF Weights
etfWeights=ob.loadETFWeights(filename)
# Get Stock Prices from the weights
ob.getStockPrices(startdate=startdate,enddate=enddate)
# ShowNa Values - Method Inherited from ETFDataCleanup
print("If ETF constituents has any NA Values")
ob.showNaColumns(ob.constituentcloseDF)
# Drop Na Values from above - Method Inherited from ETFDataCleanup
ob.constituentcloseDF=ob.dropNAColumns(ob.constituentcloseDF)
# Computre Daily Returs - Method Inherited from ETFDataCleanup
ob.changeDF=ob.computeDailyReturns(ob.constituentcloseDF)
# Weights in weights file have weights in % Clean that up
ob.stringWeightsToFloat()
# Find Net Asset Values
ob.findNetAssetValue()

###$$$ Getting data for ETF starts here
# Object to Get ETF Data
etfob=ETFStockPrices(etfticker=tickeretf)
# Get ETF Data Stock Prices
etfob.getETFTickerData(startdate=startdate,enddate=enddate)
# Show Any Na Values
print("If ETF prices has any NA Values")
etfob.showNaColumns(etfob.etfdata)
# Calculate daily returns of ETF
etfob.etfchangeDF=etfob.computeDailyReturns(etfob.etfdata['Close'])

###$$$ Calculation for Arbitrage Starts Here
# Do Calculations of ETFArbitrage
arbob=ETFArbitrage(etfob.etfchangeDF,ob.weightedStockReturns)
daysofarbitrage, tickersSTDdata=ArbitrageAnalysis().GetArbDataFrame(tickers=ob.constituentcloseDF.columns,changeDF=ob.changeDF,constituentdata=ob.constituentdata,daysofarbitrage=round(arbob.navDF.copy(),3),stdthresold=stdthresold)


###$$$ Maniputaion of data for Jinga Template Starts Here	
# Round Off NavDf and Clean up for printing
navDF=CleanDataForJinga(round(arbob.navDF.copy(),3),['Date','Close','NAV','Mispricing'],'Date',reverse=True).CleanForEndUser()
# Round Off constituentsdata and Clean up for printing
constituentsdata=CleanDataForJinga(ob.etfWeights.copy(),['Ticker','Company Name','Weights','Last','%Change','Volume'],'Ticker',reverse=False).CleanForEndUser()
# Round Off daysofarbitrage and Clean up for printing
daysofarbitrage=CleanDataForJinga(daysofarbitrage,['Date','Close','NAV','Z-Score','Mispricing'],'Date',reverse=True).CleanForEndUser()

Ticker is = XLF
Tracking Error is = 2.0


/anaconda3/lib/python3.6/site-packages/pandas_datareader/base.py:270: SymbolWarning: Failed to read symbol: 'CBOE-Z', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


If ETF constituents has any NA Values
Symbols
CBOE-Z    276
LNC         3
dtype: int64
If ETF prices has any NA Values
Series([], dtype: int64)
{'AFL':             Vol. Z-Score  Return Z-Score  Vol. Wgt Return Z-Score
Date                                                             
2019-01-03          1.09            2.72                     2.41
2019-01-04          1.61            1.24                     1.34
2019-01-07          0.27            0.50                     0.42
2019-01-08          0.12            0.26                     0.22
2019-01-09          1.35            0.30                     0.22
2019-01-10          0.59            1.26                     1.07
2019-01-11          0.75            0.25                     0.09
2019-01-14          0.46            0.49                     0.43
2019-01-15          0.87            1.01                     0.93
2019-01-16          0.40            0.86                     0.71
2019-01-17          0.31            1.08                

KeyError: "['Z-Score'] not in index"

In [5]:
tickersSTDdata

{'AFL':             Vol. Z-Score  Return Z-Score  Vol. Wgt Return Z-Score
 Date                                                             
 2019-01-03          1.09            2.72                     2.41
 2019-01-04          1.61            1.24                     1.34
 2019-01-07          0.27            0.50                     0.42
 2019-01-08          0.12            0.26                     0.22
 2019-01-09          1.35            0.30                     0.22
 2019-01-10          0.59            1.26                     1.07
 2019-01-11          0.75            0.25                     0.09
 2019-01-14          0.46            0.49                     0.43
 2019-01-15          0.87            1.01                     0.93
 2019-01-16          0.40            0.86                     0.71
 2019-01-17          0.31            1.08                     0.86
 2019-01-18          2.23            0.78                     0.98
 2019-01-22          2.78            0.75              

In [22]:
def getZScoresOnDate(date):
    ZScoresOnDate={}
    for key,value in tickersSTDdata.items():
        ZScoresOnDate[key]=dict(value.loc[date])
    return pd.DataFrame.from_dict(ZScoresOnDate).T

In [23]:
getZScoresOnDate('2019-01-03')

,Return Z-Score,Vol. Wgt Return Z-Score,Vol. Z-Score
AFL,2.72,2.41,1.09
AIG,1.58,0.63,0.24
AIZ,1.54,0.45,0.40
AJG,2.03,1.73,0.11
ALL,1.76,2.17,1.97
AMP,1.72,1.43,0.44
AON,1.38,0.44,0.26
AXP,2.25,2.35,1.29
BAC,1.20,0.95,0.67
BEN,1.12,0.70,0.16
